In [ ]:
# import cv2
from PIL import Image
import matplotlib.pyplot as plt
from inspect import getmembers, isfunction 
import argparse
import math
import random
import re

from functools import partial
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from skimage.transform import resize

from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score, cross_validate
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, roc_curve, auc, precision_recall_curve
from sklearn.utils import shuffle

import torch
import torch.nn as nn
from torch import Tensor, optim
import torchvision.transforms as T
import torchvision.transforms.functional as F
from torch.utils.data import DataLoader, random_split, Dataset, Subset
from torch.nn.modules.loss import BCEWithLogitsLoss
from torchvision.datasets import ImageFolder

from torchvision.models import vgg19, VGG19_Weights
from torchvision.models import vgg16, VGG16_Weights
from torchvision.models import resnet50, ResNet50_Weights
from torchvision.models import resnet152, ResNet152_Weights
from torchvision.models import densenet201, DenseNet201_Weights
from torchvision.models import inception_v3, Inception_V3_Weights
from torchvision.models import mobilenet_v2, MobileNet_V2_Weights

import itertools

import torchvision
# import tensorflow as tf

import xgboost as xgb

import json
import os
import cv2
import sys
from os import listdir
from os.path import splitext
from tqdm import tqdm
from pathlib import Path
import logging

import warnings

In [ ]:
def set_seed(seed = 0):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)

# Filters

In [ ]:
def create_gaborfilter():
    # This function is designed to produce a set of GaborFilters 
    # an even distribution of theta values equally distributed amongst pi rad / 180 degree
     
    filters = []
    num_filters = 16
    ksize = 35  # The local area to evaluate
    sigma = 3.0  # Larger Values produce more edges
    lambd = 10.0
    gamma = 0.5
    psi = 0  # Offset value - lower generates cleaner results
    for theta in np.arange(0, np.pi, np.pi / num_filters):  # Theta is the orientation for edge detection
        kern = cv2.getGaborKernel((ksize, ksize), sigma, theta, lambd, gamma, psi, ktype=cv2.CV_64F)
        kern /= 1.0 * kern.sum()  # Brightness normalization
        filters.append(kern)
    return filters

def apply_filter(img, filters):
# This general function is designed to apply filters to our image
     
    # First create a numpy array the same size as our input image
    newimage = np.zeros_like(img)
     
    # Starting with a blank image, we loop through the images and apply our Gabor Filter
    # On each iteration, we take the highest value (super impose), until we have the max value across all filters
    # The final image is returned
    depth = -1 # remain depth same as original image
     
    for kern in filters:  # Loop through the kernels in our GaborFilter
        image_filter = cv2.filter2D(img, depth, kern)  #Apply filter to image
         
        # Using Numpy.maximum to compare our filter and cumulative image, taking the higher value (max)
        np.maximum(newimage, image_filter, newimage)
    return newimage

def showimage(myimage, figsize=[10,10]):
    if (myimage.ndim>2):  #This only applies to RGB or RGBA images (e.g. not to Black and White images)
        myimage = myimage[:,:,::-1] #OpenCV follows BGR order, while matplotlib likely follows RGB order
         
    fig, ax = plt.subplots(figsize=figsize)
    ax.imshow(myimage, cmap = 'gray', interpolation = 'bicubic')
    plt.xticks([]), plt.yticks([])  # to hide tick values on X and Y axis
    plt.show()

# ML Functions

In [ ]:
def visualize_metrics(metrics):
    lenEpochs = len(metrics[0]) 
    epochs = [i for i in range(lenEpochs)]
    train = [( metrics[0][i]["Average Loss"]  ) for i in metrics[0]]
    val = [( metrics[1][i]["Average Loss"]  ) for i in metrics[0]]
    plt.plot(epochs, train, label = "Train")
    plt.plot(epochs, val, label = "Validation")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend() 
    
def show_metrics(metrics):
    print(f'\
    Average Loss:  {metrics["Average Loss"]:>8f},\
    Accuracy: {metrics["Accuracy"]:>0.2f}%,\
    Correct Counter: {metrics["Correct"]}/{metrics["Size"]},\
    F1 Score:{metrics["F1 Score"]: 0.2f},\
    Precision:{metrics["Precision"]: 0.2f},\
    Recall: {metrics["Recall"]: 0.2f} \n'
    )
    
    
def split(data, train_prcntg=0.8, random=True):
    # Calculate Testing percentage and round
    test_prcntg = 1 - train_prcntg
    test_prcntg = math.ceil(100 * test_prcntg) / 100
    if random==False:
        tr, tst = random_split(data, [math.ceil(len(data)*train_prcntg), math.floor(len(data)*test_prcntg)], generator=torch.Generator().manual_seed(0))
    else:
        tr, tst = random_split(data, [math.ceil(len(data)*train_prcntg), math.floor(len(data)*test_prcntg)])
    return tr, tst 


def train_test_loader(train_set, test_set, batch_size=8):
    train_loader = DataLoader(
        train_set,
        batch_size=batch_size,
    )
    test_loader = DataLoader(
        test_set,
        batch_size=batch_size,
    )
    return train_loader, test_loader


def get_indices(dataset, sample_ratio=0.2, seed=0):
    
    SAMPLE=math.floor(len(dataset) * sample_ratio)
    
    data = dataset.copy(deep=True)
    test_indices = train_indices = []
    
    # Seeded for reproducibility
    np.random.seed(0)
    
    while len(test_indices) < SAMPLE:
        
        index = np.random.randint(len(data))
        patient = data.iloc[index]["MRN"]
        
        patient_indices = data[data["MRN"] == patient].index.to_list()
        matched = dataset[dataset["MRN"] == patient].index.to_list()
            
        test_indices += matched
        
        data = data.drop(data.index[patient_indices], inplace=False).reset_index(drop=True)

    train_indices = [i for i in range(len(dataset)) if i not in test_indices]
    return train_indices, test_indices


def binary_logits_prediction(logit):
    sig = 1/(1 + np.exp(-logit))
    return classify(sig)
    
def classify(sig):
    return int(sig>=0.5)

def addFilePath(df, files):
    indx = 0
    for x in df.iterrows():
        data = x[1]
        eye = "R" if data["Eye"] == "OD" else "L"
        mrn = str(data["MRN"])
        for file in files:
            if file.startswith(mrn) and eye in file:
                df.at[indx, "Image Path"] = file
                break
        indx += 1
    
def get_htn_dataframe(path, normalize=True, standardize=True):    
    
    # Hypertension
    htn = pd.read_excel(path["HTNPath"] + r"\HTN_Age_Gender.xlsx")
    htn["Hypertension"] = 1

    # Clean Hypertension
    htn.columns = htn.columns.str.strip()
    htn["Eye"] = htn["Eye"].str.strip()

    # Add "File Path" column
    htn["Image Path"] = None
    addFilePath(htn, os.listdir(path["HTNPath"]))

    # non-Hypertension
    non_htn = pd.read_excel(path["NonHTNPath"] + r"\\NonHTN_Age_Gender.xlsx")
    non_htn["Hypertension"] = 0

    # Clean non-Hypertension
    non_htn.columns = non_htn.columns.str.strip()
    non_htn["Eye"] = non_htn["Eye"].str.strip()

    # Add "Image Path" column
    non_htn["Image Path"] = None
    addFilePath(non_htn, os.listdir(path["NonHTNPath"]))

    # Add the two df's
    df = pd.concat([htn, non_htn]) 

    # Clean again
    df = df.dropna()
    df["Gender"] = df["Gender"].str.strip()
    df.reset_index(inplace=True, drop=True)
    
    # One hot encoding for Gender 
    df["Gender"] = (df["Gender"]=="M").astype("int64")
        
    if normalize:
        df["Age"]=(df["Age"]-df["Age"].min())/(df["Age"].max()-df["Age"].min())
    if standardize:
        df["Age"] = (df["Age"] - df["Age"].mean())/df["Age"].std(ddof=0)
    
    return df
    
class HypertensionDataset(Dataset):
    def __init__(self, path, df, train_transform=None, test_transform=None):
        
        self.path = path
        
        self.train_transform = train_transform
        self.test_transform = test_transform
         
        self.df = df
        
        # Get X and y
        self.X, self.y = self.df.drop(["Hypertension", "MRN", "Eye", "Exam Date"], axis=1), self.df["Hypertension"]
        
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, idx):
        
        # Convert idx from tensor to list due to pandas bug (that arises when using pytorch's random_split)
        if isinstance(idx, torch.Tensor):
            idx = idx.tolist()

        # Get image
        image_path = self.X.iloc[idx]["Image Path"]
        if self.y.iloc[idx] == 1:
            image_path = self.path["HTNPath"] + "\\" + image_path
        else:
            image_path = self.path["NonHTNPath"] + "\\" + image_path
        img = Image.open(image_path)
        
        if self.train_transform is not None:
            img = self.train_transform(img)
        elif self.test_transform is not None:
            img = self.test_transform(img)
        
        sample = {
            "image": img,
            "features": torch.from_numpy(self.X.iloc[idx][["Age", "Gender"]].values.astype("float64")),
            "label": torch.tensor(self.y[idx]),
        }
        
        return sample
    
class TabularDataset(Dataset):
    def __init__(self, df):
            
        self.df = df
        
        self.X, self.y = self.df.drop(["Hypertension"], axis=1), self.df["Hypertension"]
        self.X.reset_index(inplace=True)
        
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, idx):
        
        sample = {
            "image": torch.tensor([]),
            "features": torch.from_numpy(self.X.iloc[idx][["Age", "Gender"]].values.astype("float64")),
            "label": torch.tensor(self.y[idx]),
        }
        
        return sample
    
    
class EarlyStopper:
    def __init__(self, patience=1, min_delta=0.001, multip=5):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_validation_loss = np.inf
        self.multip = multip
        self.trigger_stop = False

    def __call__(self, validation_loss):
        if validation_loss < self.min_validation_loss:
            self.min_validation_loss = validation_loss
            self.counter = 0
            self.trigger_stop = False
        elif validation_loss > (self.min_validation_loss + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                self.trigger_stop = True
                
    def stop(self, epoch):
        if epoch % self.multip == 0 and self.trigger_stop:
            self.trigger_stop = False
            return True
        return False
    
class ApplyTransformation(Dataset):
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform
        if transform==None:
            self.transform = T.Compose([
            T.Resize((224, 224)),
            T.ToTensor(),
            T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
            ])
        
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        sample, target = self.dataset[idx]
        sample = self.transform(sample)
        return sample, target

# Get models

In [ ]:
def get_vgg16(device="cuda:0", freeze=False, output=1):
    model = vgg16(weights=VGG16_Weights.DEFAULT)
    model = model.to(device)
    
    num_features = model.classifier[6].in_features
    features = list(model.classifier.children())[:-1]
    features.extend([nn.Linear(num_features, output)])
    model.classifier = nn.Sequential(*features)
    
    if freeze:
        for param in model.features.parameters():
            param.requires_grad = False
            
    return model

def get_vgg19(device="cuda:0", freeze=False):
    model = vgg19(weights=VGG19_Weights.DEFAULT)
    model = model.to(device)

    num_features = model.classifier[6].in_features
    features = list(model.classifier.children())[:-1]
    features.extend([nn.Linear(num_features, 1)])
    model.classifier = nn.Sequential(*features)
    
    if freeze:
        for param in model.features.parameters():
            param.requires_grad = False
    
    return model

def get_resnet50(device="cuda:0", freeze=False):
    model = resnet50(weights=ResNet50_Weights.DEFAULT)
    model = model.to(device)

    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 1)
    
    if freeze:
        model.requires_grad_(False)
        model.fc.requires_grad_(True)
    
    return model

def get_resnet152(device="cuda:0", freeze=False, with_mlp=False, outputs=1):
    model = resnet152(weights=ResNet152_Weights.DEFAULT)
    model = model.to(device)
    
    if with_mlp:
            model.fc = nn.Sequential(
            model.fc,
            nn.ReLU(inplace=True),
            nn.Linear(in_features=1000, out_features=256),
            nn.Dropout(),
            nn.ReLU(inplace=True),
            nn.Linear(in_features=256, out_features=64),
            nn.Dropout(),
            nn.ReLU(inplace=True),
            nn.Linear(in_features=64, out_features=outputs),
        )
    else:
        num_ftrs = model.fc.in_features
        model.fc = nn.Linear(num_ftrs, 1)
    
    if freeze:
        model.requires_grad_(False)
        model.fc.requires_grad_(True)
    
    return model


def get_densenet201(device="cuda:0", freeze=False, with_mlp=False, outputs=1):
    model = densenet201(weights=DenseNet201_Weights.DEFAULT)
    
    if with_mlp:
        model.classifier = nn.Sequential(
            model.classifier,
            nn.ReLU(inplace=True),
            nn.Linear(in_features=1000, out_features=256),
            nn.Dropout(),
            nn.ReLU(inplace=True),
            nn.Linear(in_features=256, out_features=64),
            nn.Dropout(),
            nn.ReLU(inplace=True),
            nn.Linear(in_features=64, out_features=outputs),
        )
    else:
        num_features = model.classifier.in_features
        features = list(model.classifier.children())[:-1]
        features.extend([nn.Linear(num_features, outputs)])
        model.classifier = nn.Sequential(*features)
        
    model = model.to(device)

    if freeze:
        for param in model.features.parameters():
            param.requires_grad = False
        
        # Unfreeze first layer
#         model.features.norm0.requires_grad_(True)    
#         model.features.conv0.requires_grad_(True)    
    
    return model

def get_mobilenetv2(device="cuda:0", freeze=False, with_mlp=False):
    model = mobilenet_v2(weights=MobileNet_V2_Weights.DEFAULT)

    if with_mlp:
        model.classifier = nn.Sequential(
            *model.classifier,
            nn.ReLU(inplace=True),
            nn.Linear(in_features=1000, out_features=256),
            nn.Dropout(),
            nn.ReLU(inplace=True),
            nn.Linear(in_features=256, out_features=64),
            nn.Dropout(),
            nn.ReLU(inplace=True),
            nn.Linear(in_features=64, out_features=1),
        )
        
    else:
        num_features = model.classifier[1].in_features
        features = list(model.classifier.children())[:-1]
        features.extend([nn.Linear(num_features, 1)])
        model.classifier = nn.Sequential(*features)
    
    model = model.to(device)
    
    if freeze:
        model.features.requires_grad_(False)
        model.classifier.requires_grad_(True)

    return model
        
def get_inceptionv3(device="cuda:0", freeze=False, with_mlp=False):
    model = inception_v3(weights=Inception_V3_Weights.DEFAULT)
    
    if with_mlp:
        model.fc = nn.Sequential(
            model.fc,
            nn.ReLU(inplace=True),
            nn.Linear(in_features=1000, out_features=256),
            nn.Dropout(),
            nn.ReLU(inplace=True),
            nn.Linear(in_features=256, out_features=64),
            nn.Dropout(),
            nn.ReLU(inplace=True),
            nn.Linear(in_features=64, out_features=1),
        )
    else:
        num_ftrs = model.fc.in_features
        model.fc = nn.Linear(num_ftrs, 1)
    
    model = model.to(device)
        
    if freeze:
        model.requires_grad_(False)
        model.fc.requires_grad_(True)

    return model